# Preamble

> This notebook contains the ground-truth copy of the code mixed with literate comments. All code in this directory is produced by running the cells in this notebook.

# Introduction

**Cortical visual prostheses** (CVPs) are devices implanted on the brain which aim to restore vision to blind people by stimulating the visual cortex.

Experiments have shown that stimulating the visual cortex produces visual percepts in both blind and sighted patients. These visual percepts are called **phosphenes** and are typically described as small round dots of colourless light like "a star in the sky." The idea behind most CVPs is to build images out of phosphenes, like how graphical displays are made out of pixels. 

But phosphenes are very limited. At the moment, we can only control the intensity of phosphenes (and, to some extent, their size). The locations of phosphenes follow the retinotopic mapping of the visual fields on the cortex, but the complexity of this mapping makes precisely controlling the positions of phosphenes in the visual field very difficult; it is much easier to instead place a regular grid of electrodes on the brain and map the phosphene locations after implantation. Other properties of phosphenes, such as their shape, individual brightness or colour, are highly variable and uncontrollable.

In addition, there are no empirical studies which describe what exactly is seen when we try to stimulate more than about five simultaneous phosphenes. The largest implant in a human used 64 electrodes, but there is inadequate description on what the implantee saw and how useful these percepts were (though the implantee purportedly was able to read large letterforms). New CVPs generally intend to stimulate on the order of hundreds of electrodes (such as 473 for the Monash Vision Group); it is not yet known what people will perceive when many electrodes are stimulate at once. Some studies have suggested that stimulating five electrodes, for example, does not produce five discrete phosphenes or a blended picture; rather, phosphenes which were stimulated by an electrode when only one electrode was stimulated completely disappear upon simultaneous stimulation as though they were not stimulated at all.

This begs the question: **how can phosphenes be used to convey useful information, given their known and unknown perceptual limitations?**

## Pattern-Based Approach

Using a pattern-based approach, we focus on **object identification**. That is, we attempt to create recognisable patterns that are matchable against object identities, and discard any location-based information.

The advantages of this approach are that:

1. By focusing on identity rather than location, stable patterns can be presented using the same phosphenes each time an object occurs in a scene. As the exact same pattern is presented each time, it may be easier to learn.

The disadvantages of this approach are that:

1. Only one object at a time can be shown.
2. If there is no location information, it may be hard to justify how this is useful in comparison to something less invasive; for example, an external camera linked to an earpiece could easily just speak out the identity of an object instead of presenting patterned vision. You could argue that presenting object identities through vision is beneficial because it doesn't take up another sensory modality, but this is a relatively small upside compared to the invasiveness of a brain implant.

## Location-Based Approach

Using a location-based approach, we focus on **object localisation**. 

## Rationale

# Phosphene Modelling

## Phosphene Rendering

In [16]:
%%writefile phosphenes.py
#!/usr/bin/env python

import numpy as np
from scipy.ndimage import gaussian_filter
import random
import math
from skimage import color

# CONSTANTS

XSIZE = 64
YSIZE = 64
PBASE = 2
SCALE = 6
EXSIZE = XSIZE // SCALE
EYSIZE = YSIZE // SCALE


def safebound(value: float, width: float, lower: float, upper: float):
    """ 
    Returns the bounded min and max about value with width.
    """
    vmin = int(max(lower, value - width))
    vmax = int(min(upper, value + width))
    return vmin, vmax

def bound(value:float, lower: float, upper:float):
    """
    Returns a bounded value.
    """
    if value > lower:
        if value < upper:
            return value
        else:
            return upper
    else:
        return lower

# Electrodes, which produce phosphenes.

class Electrode:
    def __init__(self, x: float, y: float, xsize : int = XSIZE, ysize : int = YSIZE, randomPos: float = 0):
        """
        Produces a phosphene for a single electrode.
        
        Args:
            x: float         - position in range [0, 1]. 
            y: float         - position in range [0, 1]
            randomPos: float - a scaling factor for random positioning. 
        """
        self.randomPos = randomPos
        self.x = bound(x + (random.random() - 0.5) * self.randomPos, 0, 1)
        self.y = bound(y + (random.random() - 0.5) * self.randomPos, 0, 1)
        self.xsize = xsize
        self.ysize = ysize

        self.size = PBASE * (0.5 + (4 * np.sqrt((self.x - 0.5) ** 2 + (self.y - 0.5) ** 2)) ** 2)

        self.rendered = self.render()

    def render(self):
        xmin, xmax = safebound(self.xsize * self.x, self.size, 0, self.xsize)
        ymin, ymax = safebound(self.ysize * self.y, self.size, 0, self.ysize)

        base = np.zeros((self.ysize, self.xsize))
        base[ymin:ymax, xmin:xmax] = 1

        return gaussian_filter(base, self.size)

class UniqueElectrode:
    """
    This class implements electrodes with unique characteristics such as colour and shape.
    """
    def __init__(self, x: float, y: float, xsize : int = XSIZE, ysize : int = YSIZE, randomPos: float = 0.001):
        self.x = bound(x + (random.random() - 0.5) * randomPos, 0, 1)
        self.y = bound(y + (random.random() - 0.5) * randomPos, 0, 1)
        self.size = PBASE * (0.5 + (4 * np.sqrt((self.x - 0.5) ** 2 + (self.y - 0.5) ** 2)) ** 2)
        self.colour = np.random.random(3)
        # xmod and ymod modify the shape of the phosphene
        self.xmod = 1 + (random.random()-0.5) * 3
        self.ymod = 1 + (random.random()-0.5) * 3
        self.xsize = xsize
        self.ysize = ysize

        self.rendered = self.render()

    def render(self):
        xmin, xmax = safebound(self.xsize * self.x, self.size*self.xmod, 0, self.xsize)
        ymin, ymax = safebound(self.ysize * self.y, self.size*self.ymod, 0, self.ysize)

        base = np.zeros((self.ysize, self.xsize, 3))
        base[ymin:ymax, xmin:xmax, :] = self.colour

        return gaussian_filter(base, self.size * (random.random() ** 0.3))

# Grids, which are composed of electrodes.

class RegularGrid:
    def __init__(self, exsize: int = EXSIZE, eysize: int = EYSIZE, xsize=XSIZE, ysize=YSIZE):
        """
        
        Args:
            exsize: int - x size of electrode grid 
            eysize: int - y size of electrode grid
        """
        self.exsize = EXSIZE
        self.eysize = EYSIZE
        self.grid = [
            Electrode(x / exsize, y / eysize, xsize=xsize, ysize=ysize)
            for x in range(exsize)
            for y in range(eysize)
        ]

    def render(self, values):
        product = [v * e.rendered for (v, e) in zip(values, self.grid)]
        summed = sum(product)
        summax = np.max(summed)
        return np.clip(summed, 0, 1) * 2 - 1
        # return (summed / summax) * 2 - 1

class IrregularGrid:
    def __init__(self, randomPos=2, exsize=EXSIZE, eysize=EYSIZE, xsize=XSIZE, ysize=YSIZE):
        self.exsize = EXSIZE
        self.eysize = EYSIZE
        self.grid = [
            Electrode(0.5 + (x / exsize) / 2, y / eysize, xsize=xsize, ysize=ysize, randomPos=randomPos )
            for x in range(exsize)
            for y in range(eysize)
        ]

    def render(self, values):
        product = [v * e.rendered for (v, e) in zip(values, self.grid)]
        summed = sum(product)
        summax = np.max(summed)
        return np.clip(summed, 0, 1) * 2 - 1
        # return (summed / summax) * 2 - 1

class PolarRegularGrid:
    def __init__(self, nrho, ntheta, xsize=XSIZE, ysize=YSIZE):
        self.nrho   = nrho
        self.ntheta = ntheta
        self.grid = [
            # Need to think of better way to scale.
            Electrode(((math.exp(rho**0.6) / math.exp(nrho**0.6) * math.cos((math.pi * theta / ntheta) - math.pi/2)) + 1) / 2,
                      ((math.exp(rho**0.6) / math.exp(nrho**0.6) * math.sin((math.pi * theta / ntheta) - math.pi/2)) + 1) / 2,
                      xsize = xsize,
                      ysize = ysize,
                     )
            # Ensure the central electrodes are actually visible by adding 1 to zero.
            for rho in range(1, nrho+1)
            for theta in range(ntheta)
        ]

    def render(self, values):
        product = [v * e.rendered for (v, e) in zip(values, self.grid)]
        summed = sum(product)
        summax = np.max(summed)
        return np.clip(summed, 0, 1) * 2 - 1
        # return (summed / summax) * 2 - 1
        
    def render_tensor(self, tensor):
        reshaped = tf.transpose(tf.reshape(tf.tile(tensor, tf.constant([64])), (64, 144, 1)), perm=[1, 0, 2])
        product = reshaped * self.renders
        summed = tf.reduce_sum(product, axis=0)
        return tf.clip_by_value(summed, 0, 1) * 2 - 1

class PolarRegularUniqueGrid:
    def __init__(self, nrho, ntheta, xsize=XSIZE, ysize=YSIZE):
        self.nrho   = nrho
        self.ntheta = ntheta
        self.grid = [
            # Need to think of better way to scale.
            UniqueElectrode(((math.exp(rho**0.6) / math.exp(nrho**0.6) * math.cos((math.pi * theta / ntheta) - math.pi/2)) + 1) / 2,
                            ((math.exp(rho**0.6) / math.exp(nrho**0.6) * math.sin((math.pi * theta / ntheta) - math.pi/2)) + 1) / 2,
                            xsize = xsize,
                            ysize = ysize,
                           )
            # Ensure the central electrodes are actually visible by adding 1 to zero.
            for rho in range(1, nrho+1)
            for theta in range(ntheta)
        ]

    def render(self, values):
        product = [v * e.rendered for (v, e) in zip(values, self.grid)]
        summed = sum(product)
        summax = np.max(summed)
        return np.clip(summed, 0, 1)
        # return (summed / summax) * 2 - 1
        
    def render_tensor(self, tensor):
        reshaped = tf.transpose(tf.reshape(tf.tile(tensor, tf.constant([64])), (64, 144, 1)), perm=[1, 0, 2])
        product = reshaped * self.renders
        summed = tf.reduce_sum(product, axis=0)
        return tf.clip_by_value(summed, 0, 1) * 2 - 1

class NonLinearInteractionGrid:
    def __init__(self, nrho, ntheta, xsize=XSIZE, ysize=YSIZE):
        self.nrho   = nrho
        self.ntheta = ntheta
        self.grid = [
            # Need to think of better way to scale.
            UniqueElectrode(((math.exp(rho**0.6) / math.exp(nrho**0.6) * math.cos((math.pi * theta / ntheta) - math.pi/2)) + 1) / 2,
                            ((math.exp(rho**0.6) / math.exp(nrho**0.6) * math.sin((math.pi * theta / ntheta) - math.pi/2)) + 1) / 2,
                            xsize = xsize,
                            ysize = ysize,
                           )
            # Ensure the central electrodes are actually visible by adding 1 to zero.
            for rho in range(1, nrho+1)
            for theta in range(ntheta)
        ]

    def render(self, values):
        product = [v * e.rendered for (v, e) in zip(values, self.grid)]
        summed = sum(product)
        summax = np.max(summed)
        #return np.clip(summed, 0, 1)
        return (summed / summax) * 2 - 1

        
        
# STIMULUS

class Stimulus:
    def __init__(self, image, grid, xpos=0, ypos=0):
        self.original = image
        self.shape = self.original.shape
        
        self.padder = np.zeros((3 * self.shape[0], 3 * self.shape[1]))
        self.padder[self.shape[0]:2*self.shape[0], self.shape[1]:2*self.shape[1]] = self.original
        
        self.xpos = xpos
        self.ypos = ypos
        
        self.image = self.getImage()
        
        self.grid = grid
        self.sampleWidth = 6
        
        self.vector = self.process()
            
    def get_params(self, x : float, y : float):
        
        ymin = bound(int(self.shape[0] * y - self.sampleWidth // 2), 0, self.shape[0] - 1)
        ymax = bound(int(self.shape[0] * y + self.sampleWidth // 2), 0, self.shape[0] - 1)
        xmin = bound(int(self.shape[1] * x - self.sampleWidth // 2), 0, self.shape[1] - 1)            
        xmax = bound(int(self.shape[1] * x + self.sampleWidth // 2), 0, self.shape[1] - 1)

        vals  = self.image[ymin:ymax, xmin:xmax]
        return np.mean(vals)
    
    def getImage(self):
        """ Based on xpos and ypos, get the image view from the padder.
        """
        
        xstart = self.shape[0] - int(self.xpos * self.shape[0])
        ystart = self.shape[1] - int(self.ypos * self.shape[1])
        
        return self.padder[ystart:ystart+self.shape[1], xstart:xstart+self.shape[0]]

    def process(self):
        """ Converts the stimulus into a brightness vector for the
        """

        params = [self.get_params(e.x, e.y) for e in self.grid.grid]
        return params
        #flattened = self.image.flatten(order="C")
    
    def setPos(self, xpos: float, ypos: float):
        """Translate the image. xpos and ypos lie in the range (-1, 1)
        """
        self.xpos = xpos
        self.ypos = ypos
        self.image = self.getImage()
        self.vector = self.process()
        
        
# TESTING ONLY 

import tensorflow as tf
tf.executing_eagerly()
import keras

input_shape = (72, 72)

def make_encoder_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=input_shape))
    model.add(tf.keras.layers.Dense(10*10*6))
    model.add(tf.keras.layers.Dense(10*10))
    #print(model.output_shape)
    return model

encoder = make_encoder_model()

class StimulusNet(Stimulus):
    
    def process(self):
        image_tensor = tf.convert_to_tensor(np.array([color.rgb2gray(self.image)]), dtype=tf.float32)
        return encoder(image_tensor).numpy()[0]

Writing ../phosphenes_test.py


In [17]:
%run phosphenes.py

Using TensorFlow backend.


## Digit Experiment

In [18]:
%%writefile digits.py
#!/usr/bin/env python
"""
This script runs a digit recognition psychophysics session.
"""

# # Setup

import numpy as np
import json
import phosphenes
from phosphenes import Stimulus
from datetime import datetime
from argparse import ArgumentParser
from psychopy import visual, core, gui, event
from box import Box
from psychopy.sound.backend_pygame import SoundPygame
from psychopy.tools.filetools import fromFile, toFile
from skimage import color
from imageio import imread
from random import random, choices
from PIL import Image

# I'm setting up a config dictionary with dot-syntax so it can be serialised 
# and saved with the session. I prefer explicitly keeping track of state.

config = Box({})

# Parsing the command line arguments, especially for testing.
parser = ArgumentParser(description='Digit recognition task.')

# Define command line arguments.
argspec = {
    'testing': {
        'action': 'store_const',
        'const': True,
        'default': False,
        'dest': 'testing',
        'help': 'Test the experiment and save the data.'
    },
    'ntrials': {
        'type': int,
        'nargs': '?',
        'default': 5,
        'help': 'Number of trials for the experiment.'
    },
    'ncues': {
        'type': int,
        'nargs': '?',
        'default': 10,
        'help': 'Number of cues per trial. Should be a multiple of 10 (for now) for digit stream.'
    },
    'grid': {
        'type': str,
        'nargs': '?',
        'default': 'polarRegular',
        'help': 'The grid type for rendering. One of regular, irregular, polarRegular, polarRegularUnique, or nonLinear.'
    },
    'processor': {
        'type': str,
        'nargs': '?',
        'default': 'direct',
        'help': 'The processor for the session. One of brightness of learner.'
    },
    'no-numpad': {
        'action': 'store_const',
        'const': True,
        'default': False,
        'dest': 'noNumpad',
        'help': 'Flags that normal number keys instead of numpad should be used.'
    }
}

# Add arguments to the parser.
[parser.add_argument(f'--{k}', **v) for k, v in argspec.items()]

# Parse the arguments and save into config.
args = parser.parse_args()
config.TESTING        = args.testing
config.NTRIALS        = args.ntrials
config.NCUES          = args.ncues
config.GRID_TYPE      = args.grid
config.PROCESSOR_TYPE = args.processor
config.NO_NUMPAD      = args.noNumpad


# First, we define the constants for the window size of the experiment.
# `XSIZE` and `YSIZE` refer to the size of the window on the screen.
# `EXSIZE` and `EYSIZE` refer to the size of the image data (i.e. how many 
# electrodes there are). 
# `SCALE` links the two. 

config.XSIZE  = 128
config.YSIZE  = 128
config.SCALE  = 12
config.EXSIZE = config.XSIZE // config.SCALE
config.EYSIZE = config.YSIZE // config.SCALE

# Next, we load the stimulus. Opening the image files can be expensive
# so we're doing at this at the very start and loading them into a 
# variable. 

# `IMAGE_TEMPLATE` is a string of the filepath of the stimulus digit images.
config.IMAGE_TEMPLATE = './data/digit-images-aliased/{}.png'

# `IMAGE_SIZE` is an (int, int) tuple of the image size of the first image.
# We assume that each image is of the same size as the image labelled "0"
config.IMAGE_SIZE = np.shape(imread(config.IMAGE_TEMPLATE.format(0)))  

# `IMAGE_SCALE` is an int describing the ratio of electrode size to image size.
# It assumes that EXSIZE == EYSIZE and the input images are square.
# This may need changing later. 
config.IMAGE_SCALE = config.EXSIZE / config.IMAGE_SIZE[0]  

# `IMAGES` holds the original digit images.
config.IMAGES = [np.flipud(color.rgb2gray(imread(config.IMAGE_TEMPLATE.format(digit)))) for digit in range(10)]

# `STIMULI` contains a list of numpy arrays.
# Each element in the list holds the image data (in greyscale at the moment) 
# for the digit equal to its index.
config.STIMULI = [
    np.array(Image.fromarray(image).resize((config.EXSIZE, config.EYSIZE)))
    for image in config.IMAGES
]

# We initiate a grid of electrodes.
grids = {
    'regular':   lambda: phosphenes.RegularGrid(exsize=config.EXSIZE, eysize=config.EYSIZE, xsize=config.XSIZE, ysize=config.YSIZE),
    'irregular': lambda: phosphenes.IrregularGrid(exsize=config.EXSIZE, eysize=config.EYSIZE, randomPos=0.1, xsize=config.XSIZE, ysize=config.YSIZE),
    'polarRegular': lambda: phosphenes.PolarRegularGrid(nrho=config.EXSIZE, ntheta=config.EYSIZE, xsize=config.XSIZE, ysize=config.YSIZE),
    'polarRegularUnique': lambda: phosphenes.PolarRegularUniqueGrid(nrho=config.EXSIZE, ntheta=config.EYSIZE, xsize=config.XSIZE, ysize=config.YSIZE),
    'nonLinear': lambda: phosphenes.NonLinearInteractionGrid(nrho=config.EXSIZE, ntheta=config.EYSIZE, xsize=config.XSIZE, ysize=config.YSIZE),
}

config.GRID = grids[config.GRID_TYPE]()

# We initiate the stimulus processor type.

processors = {
    'direct': phosphenes.Stimulus,
    'net': phosphenes.StimulusNet,
}

config.PROCESSOR = processors[config.PROCESSOR_TYPE]

# Templates for data paths.
config.DATETIME_FORMAT       = '%Y-%m-%d_%H-%M-%S'
config.DIGIT_SOUND_TEMPLATE  = './data/digit-voice/{}-alt.wav'

if config.TESTING:
    config.CONFIG_FILE_TEMPLATE  = './data/sessions/tests/{}_{}_config.json'
    config.SESSION_FILE_TEMPLATE = './data/sessions/tests/{}_{}_session.csv'
    config.MOUSE_FILE_TEMPLATE   = './data/sessions/tests/{}_{}_mouse.csv'
else:
    config.CONFIG_FILE_TEMPLATE  = './data/sessions/participants/{}_{}_config.json'
    config.SESSION_FILE_TEMPLATE = './data/sessions/participants/{}_{}_session.csv'
    config.MOUSE_FILE_TEMPLATE   = './data/sessions/participants/{}_{}_mouse.csv'

# Parameters for sound.
config.CORRECT_NOTE   = 'G'
config.INCORRECT_NOTE = 'Csh'
config.NOTE_DURATION  = 0.1
config.NOTE_VOLUME    = 0.5

# Session data.
config.SESSION_VARS = ['trial', 'cue', 'digit', 'keypress', 'cuetime', 'trialtime', 'sessiontime']
config.MOUSE_VARS   = ['trial', 'cue', 'digit', 'xmouse', 'ymouse', 'cuetime', 'trialtime', 'sessiontime']

# Output templates based on session data.
config.SESSION_HEADER       = ','.join(config.SESSION_VARS) + '\n'
config.SESSION_ROW_TEMPLATE = ','.join(['{' + word + '}' for word in config.SESSION_VARS]) + '\n'
config.MOUSE_HEADER         = ','.join(config.MOUSE_VARS) + '\n'
config.MOUSE_ROW_TEMPLATE   = ','.join(['{' + word + '}' for word in config.MOUSE_VARS]) + '\n'

# Mouse recording interval in seconds.
config.MOUSE_RECORD_INTERVAL = 0.2

# Text.
config.PROMPT_TEXT = "{}% complete.\n\nPress any key when ready."
config.END_TEXT    = "Thank you. \n\nPress any key to exit."

# If testing, the blank image.
if config.TESTING:
    config.BLANK_FILE = config.IMAGE_TEMPLATE.format('blank')
    config.BLANK_IMAGE = np.flipud(color.rgb2gray(imread(config.BLANK_FILE)))
    config.TEST_WINDOW_XSIZE = 480
    config.TEST_WINDOW_YSIZE = 480

# Keypress during a trial.
if config.NO_NUMPAD:
    config.KEY_LIST=[str(x) for x in range(10)]
else:
    config.KEY_LIST = ["num_" + str(x) for x in range(10)]

# When saving the config, excluding some variables due to size.
config.EXCLUDED = ['STIMULI', 'GRID', 'IMAGES', 'BLANK_IMAGE', 'PROCESSOR']


# Here, we make our main experiment, only if called from the command line.
if __name__ == "__main__":
    
    # We initiate the user details and present a dialog to the user to get those details.
    config.details = {"datetime": datetime.strftime(datetime.now(), config.DATETIME_FORMAT), "participant": ""}
    dialog         = gui.DlgFromDict(config.details, title="PROTOTYPE", fixed=["datetime"])
    
    # We interpret the dialog actions and initiate data files if proceeding.
    if dialog.OK:
        config.configFile  = config.CONFIG_FILE_TEMPLATE.format(config.details["participant"], config.details["datetime"])
        config.sessionFile = config.SESSION_FILE_TEMPLATE.format(config.details["participant"], config.details["datetime"])
        config.mouseFile = config.MOUSE_FILE_TEMPLATE.format(config.details["participant"], config.details["datetime"])
    else:
        core.quit()

    # Clocks that keep track of the experiment.
    clockSession = core.Clock()
    clockTrial   = core.Clock()
    clockCue     = core.Clock()
    mouseRecord  = core.Clock()

    # We initiate some generic sounds for correct and incorrect.
    correctSound   = SoundPygame(value=config.CORRECT_NOTE, secs=config.NOTE_DURATION)
    incorrectSound = SoundPygame(value=config.INCORRECT_NOTE, secs=config.NOTE_DURATION)
    
    correctSound.setVolume(config.NOTE_VOLUME)
    incorrectSound.setVolume(config.NOTE_VOLUME)
    
    # And we initiate the sounds for each digit.
    digitSounds = [SoundPygame(value=config.DIGIT_SOUND_TEMPLATE.format(digit)) for digit in range(10)]
    
    # Now we save the config for this session.
    with open(config.configFile, 'w+') as configFile:
        json.dump({k:v for k, v in config.items() if k not in config.EXCLUDED}, configFile)

    # We initiate a testing window if this is a testing run.
    if config.TESTING:
        testWin = visual.Window([config.TEST_WINDOW_XSIZE, config.TEST_WINDOW_YSIZE],
                                pos=(200,200), allowGUI=False, winType='pyglet')
        win = visual.Window([config.TEST_WINDOW_XSIZE, config.TEST_WINDOW_YSIZE],
                            pos=(200+config.TEST_WINDOW_XSIZE, 200), allowGUI=False, winType='pyglet', color=-1)
    else:
        # We make a window for the experiment.
        win = visual.Window(fullscr=True, allowGUI=False, winType='pyglet', color=-1)

    # Start the mouse event
    mouse = event.Mouse(visible=False, win=win)
        
    # We now start the experiment loop.
    with open(config.sessionFile, 'w+') as outfile, open(config.mouseFile, 'w+') as mousefile:

        # We first write the header of the csv file.
        outfile.write(config.SESSION_HEADER)
        mousefile.write(config.MOUSE_HEADER)

        # Start the trial loop.
        for trial in range(config.NTRIALS):

            # Set the trial clock to 0.
            # This clock will start counting from the wait screen, so includes that time..
            clockTrial.reset()
            
            # If testing, show the blank.
            if config.TESTING:
                blankStimulus = config.PROCESSOR(config.BLANK_IMAGE, config.GRID)
                rendered = config.GRID.render(blankStimulus.vector)
                imageStimulus = visual.ImageStim(testWin, image=rendered, size=(2,2))
                imageStimulus.draw(); testWin.flip()

            # Show a prompt on grey background at the beginning of the trial and wait for a keypress.
            bg     = visual.GratingStim(win, tex=None, mask=None, size=2, units='norm', color=0)
            prompt = visual.TextStim(win, text=config.PROMPT_TEXT.format(trial * 100 // config.NTRIALS))
            bg.draw(); prompt.draw(); win.flip(); event.waitKeys(clearEvents=True)

            # Create a stream of digits of length NCUES for the trial.
            stream = choices(range(10), k=config.NCUES)

            # Start the cue loop.
            for cue in range(config.NCUES):
                
                # Get a digit from the stream and initialise the stimulus.
                digit    = stream.pop()
                image    = config.IMAGES[digit]
                stimulus = config.PROCESSOR(image, config.GRID)
                
                # If this is a testing run, also draw the original image.
                if config.TESTING:
                    originalImage = visual.ImageStim(testWin, image=color.rgb2gray(image), size=(2,2))
                    originalImage.draw(); testWin.flip()
                    
                # Clear the event buffer
                event.clearEvents()      
                
                # Set the mouse to the center. Might turn off, not sure which is better.
                mouse.setPos((0,0))    
 
                # Initialise a False keypress
                keypressRaw = False
        
                # Set the cue clock to 0.
                clockCue.reset()

                # Set the mouse recording clock to 0
                mouseRecord.reset()
                
                # Loop until the keypress
                while not keypressRaw:
                    
                    # Get the mouse position and set the stimulus to the position.
                    newPos = mouse.getPos()
                    stimulus.setPos(*newPos)
                    
                    if mouseRecord.getTime() > config.MOUSE_RECORD_INTERVAL:
                    
                        mouseRow = config.MOUSE_ROW_TEMPLATE.format(
                            trial=trial,
                            cue=cue,
                            digit=digit,
                            xmouse=newPos[0],
                            ymouse=newPos[1],
                            cuetime=clockCue.getTime(),
                            trialtime=clockTrial.getTime(),
                            sessiontime=clockSession.getTime(),
                        )
                        mousefile.write(mouseRow)
                        
                        mouseRecord.reset()
                
                    # Render the stimulus
                    rendered = config.GRID.render(stimulus.vector)

                    # Create an image stimulus out of the rendered image.
                    # Then show the stimulus.
                    # Ensure stimulus is square on full screen window, assuming window has greater x dim than y dim.
                    imstim = visual.ImageStim(win, image=rendered, size = (2 * win.size[1] / win.size[0], 2))
                    imstim.draw(); win.flip()
                    
                    # Wait for a keypress. 
                    # We only need the first keypress, and want the key input from the numpage.
                    keypresses = event.getKeys(keyList = config.KEY_LIST)
                    if keypresses:
                        keypressRaw = keypresses[0]
                    #keypressRaw, *_ = event.waitKeys(clearEvents=True, keyList=config.KEY_LIST)
                
                # Check if their input was correct. 
                # Numpad keys are prepended with 'num_', so we strip it out.
                keypress = keypressRaw.strip('num_')
                correct  = (digit == int(keypress))
                
                # Create the data line.
                row = config.SESSION_ROW_TEMPLATE.format(
                    trial=trial,
                    cue=cue,
                    digit=digit,
                    keypress=keypress, 
                    cuetime=clockCue.getTime(),
                    trialtime=clockTrial.getTime(),
                    sessiontime=clockSession.getTime(),
                )
                
                # Write the data line to the session file.
                outfile.write(row)

                # Play the feedback sound.
                correctSound.play() if correct else incorrectSound.play()
                
                # Play the digit sound.
                digitSounds[digit].play()
                
        # At the end of all the trials, show an end screen and wait for key press
        # to exit.
        bg  = visual.GratingStim(win, tex=None, mask=None, size=2, units='norm', color=0)
        end = visual.TextStim(win, text=config.END_TEXT)
        bg.draw(); end.draw(); win.flip(); event.waitKeys(clearEvents=True)


Overwriting ../digits.py


In [1]:
# %run digits.py

# Generating Patterns

## Digits

### Setup

In [10]:
import glob
import imageio
import numpy as np
import os
import PIL
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import cv2
import random

from skimage import color
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from tensorflow.keras import layers, Sequential
from IPython import display

In [9]:
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"

### Creating Digits

In [67]:
def digit_to_image(digit : int):
    fig = plt.figure(figsize=(1,1))
    canvas = FigureCanvas(fig)
    ax = plt.gca()
    fig.patch.set_facecolor('black')
    plt.axis('off')
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.text(0.25 + random.random() / 2, 0.25 + random.random() / 2, str(int(digit)),
             size=48,
             color='white',
             clip_box=ax.clipbox,
             clip_on=True,
             horizontalalignment = 'center',
             verticalalignment = 'center',
             linespacing = 0,
             #bbox=dict(facecolor='red', alpha=0.5),
             transform=ax.transAxes)
    #plt.savefig(f'{digit}.png', pad_inches=0, facecolor='black')
    canvas.draw()
    width, height = fig.get_size_inches() * fig.get_dpi()
    image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
    image = (image - 127.5) / 127.5
    plt.close(fig)
    return cv2.resize(image, dsize=(64,64)).astype(np.float32)

### Training a Digit Recogniser

In [113]:
train_labels = np.random.randint(0, 10, (5000))
train_images = np.array([digit_to_image(d) for d in train_labels])

In [125]:
BUFFER_SIZE = 5000
BATCH_SIZE = 128

model = Sequential([
    layers.Conv2D(16, (4,4), padding='same', strides=(1,1), activation=tf.nn.relu, input_shape=(64, 64, 3)),
#    layers.Conv2D(16, (2,2), padding='same', strides=(1,1), activation=tf.nn.relu),
    layers.MaxPooling2D(2),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
5000/5000 [==============================] - 5s 1ms/sample - loss: 1.9070 - accuracy: 0.3668
Epoch 2/5
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.6061 - accuracy: 0.8230
Epoch 3/5
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.2158 - accuracy: 0.9444
Epoch 4/5
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.1116 - accuracy: 0.9740
Epoch 5/5
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.0556 - accuracy: 0.9914


In [119]:
test_labels = np.random.randint(0, 10, (500))
test_images = np.array([digit_to_image(d) for d in test_labels])

In [126]:
model.evaluate(test_images, test_labels)

500/500 [==============================] - 0s 384us/sample - loss: 0.0810 - accuracy: 0.9760


[0.08099063700437546, 0.976]

In [129]:
digit_aliased_model_path = "./data/models/digit-aliased.h5"
# model.save(digit_aliased_model_path)
digit_aliased_model = tf.keras.models.load_model(digit_aliased_model_path)

### Training on MNIST Digits

In [105]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

def process_images(images):
    processed = np.stack([color.gray2rgb(cv2.resize(image, dsize=(64, 64)).astype('float32')) for image in images], 0)
    processed = (processed - 127.5) / 127.5
    return processed

train_images = process_images(train_images)
test_images =  process_images(test_images)

In [107]:
BUFFER_SIZE = 60000
BATCH_SIZE = 128

model = Sequential([
    layers.Conv2D(16, (2,2), padding='same', strides=(1,1), activation=tf.nn.relu, input_shape=(64, 64, 3)),
    #layers.Conv2D(32, (2,2), padding='same', strides=(1,1), activation=tf.nn.relu),
    layers.MaxPooling2D(2),
    layers.Dropout(0.25),
    layers.Flatten(),
    #layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=2)

Epoch 1/2
60000/60000 [==============================] - 50s 835us/sample - loss: 0.2545 - accuracy: 0.9250
Epoch 2/2
60000/60000 [==============================] - 49s 817us/sample - loss: 0.1079 - accuracy: 0.9678


In [108]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 281us/sample - loss: 0.0802 - accuracy: 0.9751


[0.08023650886937976, 0.9751]

In [111]:
mnist_model_path = "./data/models/mnist.h5"
# model.save(mnist_model_path)
mnist_model = tf.keras.models.load_model(mnist_model_path)

### Putting in a Generative Network

In [134]:
digit_aliased_model.trainable = False

In [138]:
encoder = Sequential([
    digit_aliased_model,
    layers.Dense(12 * 12 * 4, use_bias=False),
    layers.LeakyReLU(),
    layers.Dropout(0.1),
    layers.Dense(12 * 12, use_bias=False)
])

In [139]:
encoder.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_16 (Sequential)   (None, 10)                164634    
_________________________________________________________________
dense_26 (Dense)             (None, 48)                480       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48)                0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 12)                576       
Total params: 165,690
Trainable params: 1,056
Non-trainable params: 164,634
_________________________________________________________________


#### Training the Generative Network

In [ ]:
cross_entropy = tf.keras.losses.CategoricalCrossentropy()

def encoder_loss(input_digit, output_digit):
    return cross_entropy(input_digit, output_digit)

encoder_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 20
num_examples_to_generate = 16

display_digits = np.array([i for i in range(10)])
display_digit_images = np.array(list(map(digit_to_image, display_digits)))

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(5,3))

    for i in range(predictions.shape[0]):
        plt.subplot(2, 5, i+1)
        plt.imshow(grid.render(predictions[i, :]), cmap='gray')
        plt.title(i)
        plt.axis('off')

    plt.savefig('../data/training_intermediate_data/training_images/generated_epoch_{:02d}.png'.format(epoch))
    plt.show()

In [ ]:
@tf.function
def train_step(digit_images):
    with tf.GradientTape() as enc_tape:
        encodings = encoder(digit_images, training=True)
        encoded_images = tf.map_fn(grid.render_tensor, encodings)
        #print(encoded_images.shape)

        output_digits = mnist_model(encoded_images)
        expected_digits = mnist_model(digit_images)

         enc_loss = encoder_loss(expected_digits, output_digits)
         # print(enc_loss)

    gradients_of_encoder = enc_tape.gradient(enc_loss, encoder.trainable_variables)
    #print(gradients_of_encoder)

    encoder_optimizer.apply_gradients(zip(gradients_of_encoder, encoder.trainable_variables))

In [ ]:
checkpoint_dir = './data/training_intermediate_data/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(encoder_optimizer=encoder_optimizer,
                                 encoder=encoder)

In [ ]:
def train(epochs):
    for epoch in range(epochs):
        start = time.time()

    for i in range(12):
        digits = np.random.randint(0, 10, (256,))
        digit_images = np.array(list(map(digit_to_image, digits)))
        train_step(digit_images)
        #print("BATCH DONE")

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(encoder,
                             epoch + 1,
                             display_digit_images
                             )

#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(encoder,
                           epochs,
                           display_digit_images)

In [ ]:
#%%time
train(EPOCHS)

# Appendix

## Generating Static Digit Images

This script generates static images of centered digits as PNG files.

In [ ]:
from PIL import Image, ImageDraw, ImageFont

baseColour = (0, 0, 0)
baseSizeX, baseSizeY = (16, 16)
textColour = (255, 255, 255)

saveExtension = "png"

for digit in range(10):
    baseImage = Image.new("RGB", (baseSizeY, baseSizeX), baseColour)
    base = ImageDraw.Draw(baseImage)

    text = str(digit)
    textSizeX, textSizeY = base.textsize(text)
    textPosition = (baseSizeX / 2 - textSizeX / 2, baseSizeY / 2 - textSizeY / 2)
    base.text(textPosition, text, textColour)

    saveName = str(digit)
    baseImage.save(f"./data/digit-images/{saveName}.{saveExtension}")

# Blank white
baseImage = Image.new("RGB", (baseSizeY, baseSizeX), (255,255,255))
base = ImageDraw.Draw(baseImage)
baseImage.save(f'./data/digit-images/blank.{saveExtension}')
